In [29]:
import pandas as pd
import requests
import json

In [30]:
with open('1_VOP.md', encoding='utf8') as file:
    text = file.read()

In [31]:
lines = text.split('\n')

text_data = []

current_question = None
current_answer = []

In [32]:
for line in lines:
    if line.startswith('## '):
        if current_question is not None:
            text_data.append([current_question, ' '.join(current_answer)])
            current_answer = []
        current_question = line[3:].strip()
    else:
        current_answer.append(line.strip())

if current_question is not None:
    text_data.append([current_question, ' '.join(current_answer)])

df = pd.DataFrame(text_data[:10], columns=['Вопрос', 'Ответ'])
df

,Вопрос,Ответ
0,Добрый день! Приводим в соответствие сведения ...,"Добрый день! Информируем, что за изменение ВРИ..."
1,Добрый день При заполнении услуги на последнем...,Добрый день! Если у вас высвечивается данный д...
2,Разрешенное использование ЗУ среднеэтажная жил...,Добрый день! За изменение ВРИ на виды предусма...
3,Что писать в графе : район субъекта Федерации ...,"Добрый день! В данном случае поле ""район субъе..."
4,Добрый день! Получила постановление об установ...,Добрый день! Вы можете уточнить у специалистов...
5,Каков порядок изменения ВРИ земельного участка...,"Добрый день! Информируем, что за изменение ВРИ..."
6,Регистрация жилого дома,Добрый день! Ваш вопрос находится вне компетен...
7,хотели бы уточнить вид разрешенного использова...,"Добрый день! Информируем, что за изменение ВРИ..."
8,В выписке из ЕГРН в сведениях о характеристика...,Добрый день! Вам следует обратиться за услугой...
9,"Земельный участок, находящийся в частной собст...","Добрый день! Информируем, что за изменение ВРИ..."


In [52]:
model = '/models/Qwen3-30B-A3B-Instruct-2507-FP8'
url = 'https://10.44.123.3/qwen30-fp8/v1/chat/completions'

def call_llm(model, url, sys_prompt, user_prompt):
    payload = {
                "model": model,
                "messages": [
                    {"role": "system", "content": sys_prompt},
                    {"role": "user", "content": user_prompt},
                ],
                "temperature": 0.1,
                "stream": False,
                "chat_template_kwargs": {"enable_thinking": False},
            }
    url = url
    response = requests.post(url=url, json=payload, verify=False).json()
    resp = response['choices'][0]['message']['content']
    return resp

In [56]:
lst = []
for _, row in df.iterrows():
    sys_prompt = '''
    Ты получаешь на вход строку pandas датафрейма. 
    Строка в первом столбце датафрейма - вопрос, который задает пользователь.
    Строка во втором столбце датафрейма - ответ, который дается на вопрос пользователя.
    Твоя задача - проанализировать вопрос и данный на него ответ и оценить его по критериям релевантности и полноты.
    Если ответ релевантный и полный, оставь его в неизменном виде.
    Если ответ нерелевантный, перепиши его, чтобы он лучше соответствовал вопросу. Можешь использовать свои знания.
    Если ответ повторяется, посмотри на удаляемые вопросы. Составь один общий вопрос, для которого подходит повторяющийся ответ. Остальные удали.
    '''
    user_prompt = f'Строка датафрейма: {row}'
    resp = call_llm(model=model, url=url, sys_prompt=sys_prompt, user_prompt=user_prompt)
    lst.append(resp)

c:\Projects\MIO\.venv\Lib\site-packages\urllib3\connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host '10.44.123.3'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
c:\Projects\MIO\.venv\Lib\site-packages\urllib3\connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host '10.44.123.3'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
c:\Projects\MIO\.venv\Lib\site-packages\urllib3\connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host '10.44.123.3'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
c:\Projects\MIO\.venv\Lib\site-packages\urllib3\connectionpool.py:1097: Insecur

In [57]:
lst

['Вопрос: Добрый день! Приводим в соответствие сведения о ВРИ  \nОтвет: Добрый день! Информируем, что за изменение ВРИ (внутреннего регистрационного номера) ответственность несёт организация, указанная в реестре. Уточните, требуется ли подтверждение изменения или оформление новых документов.',
 '**Анализ:**\n\n- **Вопрос:** "Добрый день При заполнении услуги на последнем..."  \n  → Неполный вопрос, обрывается на середине. Вероятно, пользователь хотел спросить о проблеме при заполнении услуги на последнем этапе (например, в системе госуслуг, CRM, или другой платформе).\n\n- **Ответ:** "Добрый день! Если у вас высвечивается данный д..."  \n  → Также обрывается. Ответ начинается с приветствия, но не завершён. Содержит фразу "если у вас высвечивается данный д...", что указывает на возможную ошибку или сообщение системы, но не раскрыто.\n\n---\n\n**Оценка:**\n\n- **Релевантность:** Низкая. Ответ не отвечает на вопрос, так как оба текста обрывисты и не содержат полной информации.\n- **Полнот